# Lab Session #2
## 28/08/2018
___
### Objectives:
- OLS (numpy)
- Work with Data (pandas)
    - Creation
    - Exploration
    - Visualization
- Signal processing
    - Convolution and correlation
    - Convolution of distributions
    - Correlation and covariance
    - 

In [ ]:
### CHECK YOUR PYTHON VERSION
import sys
print(sys.version)

### Packages
- sys
- warnings
- numpy
- matplotlib
- statsmodels
- pandas
- scipy
- math

#### First, let's disable warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Visualisation

Add a [magic fucntion](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#magics-explained)

In [ ]:
%matplotlib inline

In [ ]:
### First step: add packages
import numpy as np
import matplotlib.pyplot as plt

### Step 1. OLS

In [ ]:
np.random.seed()
### YOUR CODE HERE

Add column of 1's for intercept.

In [ ]:
import statsmodels.api as sm
# add constant for intercept


Use statsmodels implementation of Ordinary Least Squares.

In [ ]:
# model = 

Let's now plot the line along with the error (+/- sigma).

In [ ]:
# def line(x, a, b):
    

### Exercise 1
1. Implement closed formula solution fo the parameters of Linear Regression (from lecture notes). 
1. Implement formula for coefficient of determination (R-squared)

Feel free to use any library, as default - use [numpy.linalg](https://docs.scipy.org/doc/numpy/reference/routines.linalg.html).

Compare results with OLS method.

In [ ]:
# here goes your solution for LR parameters
import numpy.linalg as nli


In [ ]:
# here goes your solution for R-squared
SSres = 0
SStot = 0


### Step 2. Representation

**Pandas** in an open-source python package that is great for data structuring, preparation and analysis.

Features:
- Unified data representation using *DataFrame* objects - 2D heterogeneous tables with labeled axes;
- Database-like operations: join, union(append), group by, sort, column selection, filtering;
- Advanced operations: pivoting, reshaping, multi-indexing;
- Basic statistics: count, mean, std, min/max, quantiles;
- Vizualization with matplotlib.

#### 2.1. Dataframe creation
Create dataframe by reading a csv file with brain size data (analyzed [here](https://www.researchgate.net/profile/Robert_Schultz2/publication/222881180_In_vivo_brain_size_and_intelligence_Intelligence_15_223-228/links/00b4952d94932772d4000000.pdf)).
The csv contains following columns:
1. Gender: Male or Female
1. FSIQ: Full Scale IQ scores 
1. VIQ: Verbal IQ scores 
1. PIQ: Performance IQ scores 
1. Weight: body weight in pounds
1. Height: height in inches
1. MRI_Count: total pixel Count from the 18 MRI scans

In [ ]:
import pandas as pd


It is also possible to create dataset from dictionary

In [ ]:
# sin_t cos_t 


In [ ]:
# pd.DataFrame


#### 2.2. Data exploration
Let's consider simple operations to operate with dataframe.

In [ ]:
# shape

In [ ]:
# columns

In [ ]:
# 'Gender' column


You can also use boolean indexing to filter rows. For example to compute the *mean value* of verbal IQ score for females it is possible to use the following line:

In [ ]:
# mean data[]['VIQ'] for data['Gender'] == 'Female' 


To get simple statistics for each column use `descibe` function:

In [ ]:
# describe()


The `groupby` operation allows to split dataframe into groups. 

It returns `GroupBy` object which can be used to apply a function for each group separately ([more](https://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook-grouping) documentation on grouping).

In [ ]:
# mean for groupby_gender 


In [ ]:
# groupby_gender.mean()

### Exercise 2
	•	What is the mean value for VIQ for the full population?
	•	How many males/females were included in this study? 
	•	What is the average value of MRI counts expressed in log units, for males and females?
**Hint**: use ‘tab completion’ to find out the methods that can be called, instead of `mean` in the above example.

In [ ]:
# print mean VIQ for the full population


In [ ]:
# print the numbers of males and females


In [ ]:
# print average MRI counts in log units for each gender



#### 2.3. Data visualization
Let's use pandas plotting to make simple boxplots and 3x3 matrix of scatter plots for 3 columns.

In [ ]:
# Box plots of different columns for each gender


In [ ]:
#from pandas.tools.plotting import scatter_matrix
#scatter_matrix(data[['Weight', 'Height', 'MRI_Count']])


In [ ]:
#scatter_matrix(data[['FSIQ', 'VIQ', 'PIQ']])
#plt.show()

### Exercise 3

Plot the scatter matrix for males only, and for females only. 

In [ ]:
# scatter matrix for males


In [ ]:
# scatter matrix for females 


### 3. Signal processing

#### 3.1 Convolution and correlation

From the point of view of signal processing convolution and correlation are very similar operations. They are defined as following

Convolution:
$$ (f*g)(y) = \int\limits_{-\infty}^{\infty} f(x)g(y-x)dx$$
Cross-correlation:
$$ (f\star g)(y) = \int\limits_{-\infty}^{\infty} f(x)g(y+x)dx$$

If `g(x)` is symmetric then the convolution and cross-correlation are the same.

Let's create a numpy array with square-shaped signal.

In [ ]:
N = 100 # sample count
startX = 40  
endX = 60
x = np.arange(N)
x = ((x < endX) * (x> startX))*10
plt.plot(x);

Let's create a triangular signal.

In [ ]:
y = np.zeros(100)
heightY = 1.0
startY = 20
endY=80
widthY = (endY-startY)/2.0
for i in range(startY,int(N/2)):
 y[i] =  (i-startY) * (heightY / widthY) 
for i in range(int(N/2),endY):
 y[i] = 2*heightY - ((i-startY) * (heightY / widthY))
plt.plot(y);

Now let's apply two operations on the inputs and compare results.

In [ ]:
from scipy.signal import convolve
x_conv = convolve(x,y,'same')
plt.plot(x_conv);

In [ ]:
from scipy.signal import correlate
x_corr = correlate(x,y,'same')
plt.plot(x_corr);

As you see, the result of convolution for symmetric kernel is the same as result of correlation.

### Exercise 4
Try these two operations with the non-symetric kernel.

In [ ]:
# your code here


#### 3.2. Convolution of distributions
In statistics convolution plays important role.

The distribution of the *sum of two independent variables* is the convolution of the distributions.

I.e. if we consider two independent continuous random variables `X` and `Y` with PDFs `f` and `g` the PDF of their sum (X+Y) will be computed as: 

$$ d(x) = (f * g)(x) $$

We can get an interesting result if we consider the sum of several random variables with identical distributions, i.e. convolution of a distribution with itself several times.
2
For example, let's take a uniform distribution.

In [ ]:
%%capture
orig = np.random.uniform(0,10,1200)
h = plt.hist(orig, bins=120, normed=True);
orig = h[0]
t = h[1]
xconv = np.copy(orig)

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(2, 3, 1)
ax.set_title("f")
ax.plot(t[:-1], orig);
for i in range(5):
    xconv = convolve(orig, xconv)
    ax = plt.subplot(2, 3, i+2)
    ax.set_title(("f*"*(i+2))[:-1])
    ax.plot(xconv);

This is an illustration of Central Limit Theorem: the distribution of the sum of random variables which are iid (independent identically distributed) has a form of a Gaussian.

#### 3.3 Correlation and covariance

In statistics the linear correlation between two variables X and Y can be measured using Pearson correlation coefficient:

$$ corr(x,y) = \frac{\sum\limits_{i=1}^n(x_i-\bar{x})(y_i-\bar{y})}{\sqrt {\sum\limits_{i=1}^n(x_i-\bar{x})^2\sum\limits_{i=1}^n(y_i-\bar{y})^2}}$$

In [ ]:
import pandas as pd
data = pd.read_csv('examples/brain_size.csv', sep=';', na_values=".")
data.fillna(data.mean(), inplace=True)
data.corr()

The unnormalized version of the correlation coefficient is the measure of joint variability - **covariance**.
$$ cov(x,y) = \frac{1}{n-1}\sum\limits_{i=1}^n(x_i-\bar{x})(y_i-\bar{y}) $$

In [ ]:
data.cov()

As you can see, the correlation and covariance always have the same sign which shows the tendency in the linear relationship between variables.

Let's now consider more closely the correlation between `Height` and `Weight` of the people from experiments.

In [ ]:
import scipy 
scipy.stats.pearsonr(data['Height'], data['Weight'])[0]

In [ ]:
plt.scatter(data['Height'], data['Weight']);

### Exercise 5

Let's compare the difference in the definition of **correlation** from the signal processing point of view and from the point of view of statistics.

1. Compute the correlation from `scipy.signal` package of demeaned signals. Divide by the number of observations - 1.
2. Compute the covariance using `numpy` or any other package. Compare results

In [ ]:
# compute demeaned correlation divided by n-1 using scipy.signal


In [ ]:
# compute covariance using numpy


### Exercise 6

1. Implement the computation of Pearson correlation coefficient. 
1. Implement calculation of **t-statistics**, and **p-value** for the correlation coefficient. Add p-value to return value of the function.
1. Compare results with `scipy` implementation for `Height` and `Weight`.

In [ ]:
# from math import sqrt
# def pearsonr(x, y):